In [23]:
#!pip install evaluate
#!pip install datasets
!pip install seqeval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [4]:
import evaluate
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification

2024-04-23 08:15:23.023147: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-23 08:15:23.023286: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-23 08:15:23.150899: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
ner_datasets = load_dataset("msra_ner", cache_dir="./data")

Generating train split:   0%|          | 0/45001 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3443 [00:00<?, ? examples/s]

In [6]:
ner_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 45001
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3443
    })
})

In [7]:
print(ner_datasets["train"][0])

{'id': '0', 'tokens': ['当', '希', '望', '工', '程', '救', '助', '的', '百', '万', '儿', '童', '成', '长', '起', '来', '，', '科', '教', '兴', '国', '蔚', '然', '成', '风', '时', '，', '今', '天', '有', '收', '藏', '价', '值', '的', '书', '你', '没', '买', '，', '明', '日', '就', '叫', '你', '悔', '不', '当', '初', '！'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [8]:
ner_datasets["train"].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}

In [9]:
label_list = ner_datasets["train"].features["ner_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

In [10]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

In [11]:
tokenizer(ner_datasets["train"][0]["tokens"], is_split_into_words=True)   # 对于已经做好tokenize的数据，要指定is_split_into_words参数为True

{'input_ids': [101, 2496, 2361, 3307, 2339, 4923, 3131, 1221, 4638, 4636, 674, 1036, 4997, 2768, 7270, 6629, 3341, 8024, 4906, 3136, 1069, 1744, 5917, 4197, 2768, 7599, 3198, 8024, 791, 1921, 3300, 3119, 5966, 817, 966, 4638, 741, 872, 3766, 743, 8024, 3209, 3189, 2218, 1373, 872, 2637, 679, 2496, 1159, 8013, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [12]:
print(tokenizer(ner_datasets["train"][0]["tokens"], is_split_into_words=True).word_ids())

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, None]


In [13]:
res = tokenizer("interesting word")
res

{'input_ids': [101, 10673, 12865, 12921, 8181, 8681, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [14]:
res.word_ids()

[None, 0, 0, 0, 0, 1, None]

In [15]:
# 借助word_ids 实现标签映射
def process_function(examples):
    tokenized_exmaples = tokenizer(examples["tokens"], max_length=128, truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_exmaples.word_ids(batch_index=i)
        label_ids = []
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_id])
        labels.append(label_ids)
    tokenized_exmaples["labels"] = labels
    return tokenized_exmaples

这段代码是一个处理函数，用于对给定的例子进行标记映射。它接受一个包含文本和相应命名实体识别（NER）标签的字典作为输入，并返回一个包含标记化文本和映射标签的字典。以下是该函数的详细解释：
1. `tokenizer`：这是一个标记器函数，用于将文本转换为标记。它接受文本、最大长度、截断标志和是否按单词分割的标志作为输入。
2. `tokenized_exmaples = tokenizer(examples["tokens"], max_length=128, truncation=True, is_split_into_words=True)`：这行代码使用标记器函数对输入文本进行标记化处理，并将结果存储在`tokenized_exmaples`变量中。最大长度设置为128，如果文本长度超过128，则进行截断。同时，标记器会按单词进行分割。
3. `labels = []`：这是一个空列表，用于存储映射后的标签。
4. `for i, label in enumerate(examples["ner_tags"]):`：这行代码遍历输入字典中的NER标签，并为每个标签分配一个索引`i`。
5. `word_ids = tokenized_exmaples.word_ids(batch_index=i)`：这行代码获取标记化文本中的单词ID。`word_ids`是一个列表，其中包含与每个标记对应的单词ID。如果标记是一个子单词，则其单词ID与其前一个标记的单词ID相同；如果标记是一个特殊标记（如CLS或SEP），则其单词ID为None。
6. `label_ids = []`：这是一个空列表，用于存储映射后的标签ID。
7. `for word_id in word_ids:`：这行代码遍历`word_ids`列表中的每个单词ID。
8. `if word_id is None:`：这行代码检查单词ID是否为None。如果是，说明当前标记是一个特殊标记，我们将标签ID设置为-100。
9. `else:`：如果单词ID不是None，说明当前标记是一个单词的一部分。我们将输入标签中的相应标签ID添加到`label_ids`列表中。
10. `labels.append(label_ids)`：这行代码将`label_ids`列表添加到`labels`列表中。
11. `tokenized_exmaples["labels"] = labels`：这行代码将`labels`列表添加到标记化示例字典中。
12. `return tokenized_exmaples`：这行代码返回包含标记化文本和映射标签的字典。
总之，这个处理函数使用标记器对输入文本进行标记化处理，并将输入标签映射到标记化文本上。映射后的标签将用于后续的命名实体识别任务。


In [16]:
tokenized_datasets = ner_datasets.map(process_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/45001 [00:00<?, ? examples/s]

Map:   0%|          | 0/3443 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 45001
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3443
    })
})

In [17]:
print(tokenized_datasets["train"][5])

{'id': '5', 'tokens': ['我', '们', '是', '受', '到', '郑', '振', '铎', '先', '生', '、', '阿', '英', '先', '生', '著', '作', '的', '启', '示', '，', '从', '个', '人', '条', '件', '出', '发', '，', '瞄', '准', '现', '代', '出', '版', '史', '研', '究', '的', '空', '白', '，', '重', '点', '集', '藏', '解', '放', '区', '、', '国', '民', '党', '毁', '禁', '出', '版', '物', '。'], 'ner_tags': [0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 0, 0, 0, 0, 0, 0], 'input_ids': [101, 2769, 812, 3221, 1358, 1168, 6948, 2920, 7195, 1044, 4495, 510, 7350, 5739, 1044, 4495, 5865, 868, 4638, 1423, 4850, 8024, 794, 702, 782, 3340, 816, 1139, 1355, 8024, 4730, 1114, 4385, 807, 1139, 4276, 1380, 4777, 4955, 4638, 4958, 4635, 8024, 7028, 4157, 7415, 5966, 6237, 3123, 1277, 510, 1744, 3696, 1054, 3673, 4881, 1139, 4276, 4289, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [18]:
# 对于所有的非二分类任务，切记要指定num_labels，否则就会device错误
model = AutoModelForTokenClassification.from_pretrained("bert-base-chinese", num_labels=len(label_list))


model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
model.config.num_labels

7

In [24]:
# 这里方便大家加载，替换成了本地的加载方式，无需额外下载
seqeval = evaluate.load("seqeval")
seqeval

EvaluationModule(name: "seqeval", module_type: "metric", features: {'predictions': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence')}, usage: """
Produces labelling scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: List of List of predicted labels (Estimated targets as returned by a tagger)
    references: List of List of reference labels (Ground truth (correct) target values)
    suffix: True if the IOB prefix is after type, False otherwise. default: False
    scheme: Specify target tagging scheme. Should be one of ["IOB1", "IOB2", "IOE1", "IOE2", "IOBES", "BILOU"].
        default: None
    mode: Whether to count correct entity labels with incorrect I/B tags as true positives or not.
        If you want to only count exact matches, pass mode="strict". default: None.
    sample_weight: Array-like of sha

In [25]:
import numpy as np

def eval_metric(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=-1)

    # 将id转换为原始的字符串类型的标签
    true_predictions = [
        [label_list[p] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels) 
    ]

    true_labels = [
        [label_list[l] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels) 
    ]

    result = seqeval.compute(predictions=true_predictions, references=true_labels, mode="strict", scheme="IOB2")

    return {
        "f1": result["overall_f1"]
    }
    

这段代码定义了一个评估指标函数`eval_metric`，用于评估序列标注模型在命名实体识别（NER）任务上的性能。函数的输入是一个元组`pred`，其中包含模型预测的分数和真实的标签。代码使用`numpy`库来处理数组运算，并使用`seqeval`库来计算评估指标。
以下是代码的详细解释：
1. `import numpy as np`：导入`numpy`库，用于执行高效的数学运算。
2. `def eval_metric(pred):`：定义一个名为`eval_metric`的函数，它接受一个参数`pred`，这是一个包含模型预测分数和真实标签的元组。
3. `predictions, labels = pred`：将`pred`元组解包为两个变量`predictions`和`labels`，分别存储模型预测的分数和真实的标签。
4. `predictions = np.argmax(predictions, axis=-1)`：使用`numpy`的`argmax`函数沿最后一个轴（即类别轴）找到每个样本的最大预测分数的索引，这些索引对应于预测的标签ID。
5. `true_predictions = [...]`：这是一个列表推导式，用于将预测的标签ID转换为原始的字符串类型的标签。对于每个预测和标签序列，它遍历它们并创建一个新的列表，其中只包含标签不是-100的元素。`label_list[p]`用于将标签ID转换为字符串标签。
6. `true_labels = [...]`：这是另一个列表推导式，与`true_predictions`类似，但它用于转换真实的标签ID为字符串标签。
7. `result = seqeval.compute(predictions=true_predictions, references=true_labels, mode="strict", scheme="IOB2")`：这行代码使用`seqeval`库的`compute`函数来计算评估指标。`true_predictions`是模型的预测，`true_labels`是真实的标签。`mode`参数设置为"strict"，表示严格评估模式，`scheme`参数设置为"IOB2"，表示使用IOB2标签格式。
8. `return {"f1": result["overall_f1"]}`：函数返回一个字典，其中包含整体的F1分数，这是NER任务中常用的评估指标。
总之，这个函数用于评估模型在NER任务上的性能，它将模型输出的分数转换为标签，并使用`seqeval`库来计算F1分数。


In [26]:
args = TrainingArguments(
    output_dir="models_for_ner",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="f1",
    load_best_model_at_end=True,
    logging_steps=50,
    num_train_epochs=1,
    report_to=['tensorboard']
)

In [ ]:
from accelerate import Accelerator
from accelerate.data_loader import DataLoaderConfiguration

# 创建一个DataLoaderConfiguration实例并设置相应的参数
dataloader_config = DataLoaderConfiguration(
    dispatch_batches=None,
    split_batches=False,
    even_batches=True,
    use_seedable_sampler=True
)

# 现在使用DataLoaderConfiguration实例来初始化Accelerator
accelerator = Accelerator(dataloader_config=dataloader_config)


In [27]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=eval_metric,
    data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer)
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [28]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,0.020900,0.020928,0.946303


TrainOutput(global_step=704, training_loss=0.04033736163877289, metrics={'train_runtime': 507.8956, 'train_samples_per_second': 88.603, 'train_steps_per_second': 1.386, 'total_flos': 2832423606981672.0, 'train_loss': 0.04033736163877289, 'epoch': 1.0})

In [29]:
trainer.evaluate(eval_dataset=tokenized_datasets["test"])

{'eval_loss': 0.02092777192592621,
 'eval_f1': 0.9463030643800956,
 'eval_runtime': 16.7905,
 'eval_samples_per_second': 205.056,
 'eval_steps_per_second': 1.608,
 'epoch': 1.0}

In [30]:
from transformers import pipeline

# 使用pipeline进行推理，要指定id2label
model.config.id2label = {idx: label for idx, label in enumerate(label_list)}
model.config

BertConfig {
  "_name_or_path": "bert-base-chinese",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "

In [31]:
# 如果模型是基于GPU训练的，那么推理时要指定device
# 对于NER任务，可以指定aggregation_strategy为simple，得到具体的实体的结果，而不是token的结果
ner_pipe = pipeline("token-classification", model=model, tokenizer=tokenizer, device=0, aggregation_strategy="simple")

In [32]:
res = ner_pipe("马云在杭州创建了阿里巴巴")
res

[{'entity_group': 'PER',
  'score': 0.9968899,
  'word': '马 云',
  'start': 0,
  'end': 2},
 {'entity_group': 'LOC',
  'score': 0.99767697,
  'word': '杭 州',
  'start': 3,
  'end': 5},
 {'entity_group': 'ORG',
  'score': 0.98138344,
  'word': '阿 里 巴 巴',
  'start': 8,
  'end': 12}]

In [33]:
# 根据start和end取实际的结果
ner_result = {}
x = "马云在杭州创建了阿里巴巴"
for r in res:
    if r["entity_group"] not in ner_result:
        ner_result[r["entity_group"]] = []
    ner_result[r["entity_group"]].append(x[r["start"]: r["end"]])

ner_result

{'PER': ['马云'], 'LOC': ['杭州'], 'ORG': ['阿里巴巴']}